# Simple test: interactive plot of example mesh

In [ ]:
import pyvista as pv
# pv.set_jupyter_backend('server')
# pv.start_xvfb()

In [ ]:
print(pv.Report())

In [ ]:
from pyvista import examples
mesh = examples.download_dragon()
mesh['scalars'] = mesh.points[:, 1]
mesh.plot(cpos='xy', cmap='plasma')

# More complicated test: render cloud GIF

In [ ]:
import os, glob
from pathlib import Path
import xarray as xr
import pyvista as pv
import pvxarray

In [ ]:
folder = '/glade/derecho/scratch/joko/from_cheyenne_pt1/cm1-iceball-ppe/iceball-test-ppe-n20/out/1' # folder of netcdf files, each file is one time step
ds_list = []
for f in sorted(glob.glob(os.path.join(folder, 'cm1out_0*'))):
    f_temp = f.split(".",1)[0].rsplit("_", 1)[1]
    f_int = int(f_temp)
    if (f_int >= 10) & (f_int < 15): # subset of time slices (optional)
        # print(f)
        ds = xr.open_dataset(f)
        variables = ['qi1', 'qi2', 'qi3', 'qi4']
        ds_qi = ds[variables]
        # ds_mean = ds_qi.mean(dim='yh')
        ds_list.append(ds_qi)
    
ds = xr.concat(ds_list, dim = 'time')
ds 

In [ ]:
ds_new = ds.assign(qi=ds['qi1']+ds['qi2']+ds['qi3']+ds['qi4'])
ds_new

In [ ]:
# test with single frame 
ds_slice = ds_new.isel(time=0)
da = ds_slice.qi
mesh = da.pyvista.mesh(x='xh', y='yh', z='zh')
mesh.plot(opacity='linear', cmap='binary', volume=True, clim = [0, 0.0002])

In [ ]:
### === Create GIF from 10 frames === 
# Create plotter
ds_slice = ds_new.isel(time=0)
da = ds_slice.qi
mesh = da.pyvista.mesh(x='xh', y='yh', z='zh')
plotter = pv.Plotter(notebook=False, off_screen=True, window_size=[1024, 1024])
n_frames = len(ds_new.time)
plotter.add_volume(
    mesh,
    opacity='linear',
    cmap="binary",
    show_scalar_bar=False,
    clim=[0, 0.0002],
)

plotter.open_gif("iceball-3d-viz-test.gif")

# render and don't close
plotter.show(auto_close=False)
plotter.write_frame()

# update each frame
for i in range(1, n_frames):
    print(f'Actors: {plotter.actors}')
    print(f'loop {i}')
    ds_time_i = ds_new.isel(time=i)
    da_qi = ds_time_i.qi
    mesh.point_data['qi'] = da_qi.to_numpy().flatten()
    # print(mesh.active_scalars_info)
    # print('removing actor')
    removed = plotter.clear_actors()
    # print(f'removed? {removed}')
    # print(f'Actors: {plotter.actors}')
    current_scalar = f'qi_{i}'
    actor = plotter.add_volume(
        mesh,
        opacity='linear',
        cmap="binary",
        show_scalar_bar=False,
        clim=[0, 0.0002],
        render=True
    )
    plotter.write_frame()
    # plotter.screenshot(f'screenshot_{i}.png')

plotter.close()